In [1]:
import funciones_comunes
import funciones_mongo
import importlib
importlib.reload(funciones_comunes) # recarga el módulo después de modificarl
importlib.reload(funciones_mongo) # recarga el módulo después de modificarl


<module 'funciones_mongo' from '/work/funciones_mongo.py'>

In [2]:
# info = {"usuarios" : [], "destinos" : [], "hoteles": [], "actividades":[], "reservas":[] }
info = funciones_comunes.procesar_datos()
print(info)


{'usuarios': [{'usuario_id': 1, 'nombre': 'María Pérez', 'email': 'maria.perez@example.com', 'telefono': '+54 11 4567 1234'}, {'usuario_id': 2, 'nombre': 'Juan López', 'email': 'juan.lopez@example.com', 'telefono': '+54 221 334 5566'}, {'usuario_id': 3, 'nombre': 'Carla Gómez', 'email': 'carla.gomez@example.com', 'telefono': '+54 261 789 2233'}, {'usuario_id': 4, 'nombre': 'Luis Fernández', 'email': 'luis.fernandez@example.com', 'telefono': '+54 299 444 9988'}, {'usuario_id': 5, 'nombre': 'Ana Torres', 'email': 'ana.torres@example.com', 'telefono': '+54 381 123 4567'}], 'destinos': [{'destino_id': 1, 'ciudad': 'Bariloche', 'pais': 'Argentina', 'tipo': 'Montaña', 'precio_promedio': 90000}, {'destino_id': 2, 'ciudad': 'Cancún', 'pais': 'México', 'tipo': 'Playa', 'precio_promedio': 150000}, {'destino_id': 3, 'ciudad': 'Madrid', 'pais': 'España', 'tipo': 'Cultural', 'precio_promedio': 110000}, {'destino_id': 4, 'ciudad': 'Roma', 'pais': 'Italia', 'tipo': 'Histórico', 'precio_promedio': 100

#### insertar info en mongo 

In [3]:
funciones_mongo.insertar_varios_documentos("usuarios",info["usuarios"])

In [4]:
funciones_mongo.insertar_varios_documentos("destinos",info["destinos"]) 

In [5]:
funciones_mongo.insertar_varios_documentos("hoteles",info["hoteles"]) 

In [6]:
funciones_mongo.insertar_varios_documentos("actividades",info["actividades"]) 

In [7]:
funciones_mongo.insertar_varios_documentos("reservas",info["reservas"]) 

In [8]:
for elem in funciones_mongo.db.hoteles.find():
    print(elem) 
print('')
for elem in funciones_mongo.db.destinos.find():
    print(elem) 
print('')
for elem in funciones_mongo.db.usuarios.find():
    print(elem)
print('')
for elem in funciones_mongo.db.actividades.find():
    print(elem) 
print('')
for elem in funciones_mongo.db.reservas.find():
    print(elem) 

{'_id': ObjectId('68f407c675fb4b42a862051c'), 'hotel_id': 1, 'nombre': 'Hotel Sol', 'ciudad': 'Bariloche', 'precio': 85000, 'calificacion': 4, 'servicios': ['wifi', 'pileta', 'desayuno']}
{'_id': ObjectId('68f407c675fb4b42a862051d'), 'hotel_id': 2, 'nombre': 'Cumbres Andinas', 'ciudad': 'Bariloche', 'precio': 120000, 'calificacion': 5, 'servicios': ['wifi', 'spa', 'pileta']}
{'_id': ObjectId('68f407c675fb4b42a862051e'), 'hotel_id': 3, 'nombre': 'Altos del Norte', 'ciudad': 'Jujuy', 'precio': 60000, 'calificacion': 3, 'servicios': ['wifi']}
{'_id': ObjectId('68f407c675fb4b42a862051f'), 'hotel_id': 4, 'nombre': 'Montaña Real', 'ciudad': 'Mendoza', 'precio': 95000, 'calificacion': 4, 'servicios': ['wifi', 'pileta']}
{'_id': ObjectId('68f407c675fb4b42a8620520'), 'hotel_id': 5, 'nombre': 'Estancia Colonial', 'ciudad': 'Córdoba', 'precio': 70000, 'calificacion': 4, 'servicios': ['wifi', 'desayuno']}
{'_id': ObjectId('68f4153b2f4e3779e8b2a05c'), 'hotel_id': 1, 'nombre': 'Hotel Sol', 'ciudad':

Base de datos:

viajes ← (esa es la variable db) 
Colecciones dentro de viajes:

usuarios

destinos

hoteles

actividades

reservas

Ejemplo visual:
viajes
 ├── usuarios
 │    ├── { "usuario_id": 1, "nombre": "Juan", "email": "juan@mail.com" }
 │    └── { "usuario_id": 2, "nombre": "Ana", "email": "ana@mail.com" }
 ├── destinos
 │    ├── { "destino_id": 1, "nombre": "Roma" }
 │    └── { "destino_id": 2, "nombre": "Tokio" }
 ├── hoteles
 │    └── { "hotel_id": 1, "nombre": "Hilton", "destino_id": 1 }
 ├── actividades
 │    └── { "actividad_id": 1, "nombre": "Tour guiado", "destino_id": 1 }
 └── reservas
      └── { "reserva_id": 1, "usuario_id": 1, "hotel_id": 1, "fecha": "2025-10-18" }

In [9]:
# db.destinos.drop() # Elimina por completo la colección destinos dentro de la base de datos viajes.

In [10]:
from funciones_comunes import procesar_datos_neo4j
from funciones_neo4j import insertar_varios_nodos, insertar_varias_relaciones

# Cargar nodos
usuarios = procesar_datos_neo4j("usuarios.csv")
insertar_varios_nodos("Usuario", usuarios)

destinos = procesar_datos_neo4j("destinos.csv")
insertar_varios_nodos("Destino", destinos)

# Cargar relaciones
visitas = procesar_datos_neo4j("visitas.csv")
# Convertir CSV a formato que usa insertar_varias_relaciones
rel_visitas = [
    {
        "label_origen": "Usuario",
        "prop_origen": "usuario_id",
        "valor_origen": v["usuario_id"],
        "label_destino": "Destino",
        "prop_destino": "destino_id",
        "valor_destino": v["destino_id"],
        "tipo": "VISITO"
    }
    for v in visitas
]
insertar_varias_relaciones(rel_visitas)

FileNotFoundError: [Errno 2] No such file or directory: 'usuarios.csv'